<a href="https://colab.research.google.com/github/denncraft/AI_ML_notes/blob/main/Deep_Learning_2023_chapter_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

возвращаясь к примеру из chapter 2, где была написана модель  и класс NaiveDense

если его переписать, базирруясь на классе keras -> layer

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# класс наследуется от класса `Layer`
class SimpleDense(keras.layers.Layer):
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation

  # веса создаются в метоже build()
  def build(self, input_shape):
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
    self.b = self.add_weight(shape=(self.units,), initializer="zeros")

  # вычисления прямого прохода определяются в метоже call()
  def call(self, inputs):
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y


теперь создание слоя будет выглядеть так:

причем при создании слоя мы определяем только его выходной размер, а размерность входных тензоров слоя определит сам и создаст соответствующей размерности тензоры для хранения коэффициентов.
при этом мы даже не вызываем метод build(), он автоматически вызывается методом __call__.

In [7]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2,784)) # к примеру всего 2 элемента, но мы помним, что такая формы получалась после слоя flaten из (2, 28, 28)
output_tensor = my_dense(input_tensor)

print(output_tensor.shape)

(2, 32)


#сети бывают:

- сети с двумя ветвями (two-branches networks)
- многоголовые сети (multihead networks)
- входные блоки (inception blocks)

#оптимизаторы:
- SGD (с импульсом или без)
- RMSprop
- Adam
- Adagrad
- ...

#функции потерь:
- CategoricalCrossentropy
- SparseCategoricalCrossEntropy
- BinaryCrossentropy
- MeanSquareError
- KLDivergence
- CosineSimilitary
- ...

#метрики:
- CategicalAccuracy
- SparceCategoricalAccuracy
- BinaryAccuracy
- AUC
- Preccision
- Recall
- ...



In [ ]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(
    mean=[0, 3],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(
    mean=[3, 0],
    cov=[[1, 0.5],[0.5, 1]],
    size=num_samples_per_class)
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1), dtype="float32"),
                     np.ones((num_samples_per_class, 1), dtype="float32")))

разделим данные на две части 30/70 для обучение/валидации

и перемешаем данные, чтобы данные для обучения были и не из начала и не из конца датасета,  а в перемешку

обучим модель


In [24]:
model = keras.Sequential([keras.layers.Dense(1)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.05),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,
    training_targets,
    epochs=10,
    batch_size=16,
    validation_data=(val_inputs, val_targets)
)

Epoch 1/10
88/88 [==============================] - 1s 7ms/step - loss: 0.6135 - binary_accuracy: 0.9907 - val_loss: 0.0248 - val_binary_accuracy: 0.9983
Epoch 2/10
88/88 [==============================] - 0s 5ms/step - loss: 0.0382 - binary_accuracy: 0.9950 - val_loss: 0.0506 - val_binary_accuracy: 0.9983
Epoch 3/10
88/88 [==============================] - 0s 4ms/step - loss: 0.0380 - binary_accuracy: 0.9921 - val_loss: 0.0264 - val_binary_accuracy: 0.9983
Epoch 4/10
88/88 [==============================] - 0s 4ms/step - loss: 0.0368 - binary_accuracy: 0.9971 - val_loss: 0.0330 - val_binary_accuracy: 0.9983
Epoch 5/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0403 - binary_accuracy: 0.9871 - val_loss: 0.1446 - val_binary_accuracy: 0.8500
Epoch 6/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0365 - binary_accuracy: 0.9943 - val_loss: 0.0269 - val_binary_accuracy: 0.9983
Epoch 7/10
88/88 [==============================] - 0s 2ms/step - loss: 0.03

In [25]:
predictions = model.predict(val_inputs, batch_size=128)
print(predictions[:10])

5/5 [==============================] - 0s 2ms/step
[[-0.04030961]
 [ 1.1462681 ]
 [-0.01229393]
 [-0.12701714]
 [ 1.1972651 ]
 [ 0.74778813]
 [-0.18762553]
 [ 1.0157523 ]
 [ 0.10992399]
 [ 0.73033553]]
